In [22]:
import numpy as np
import pandas as pd
import openai
import numpy as np
from numpy import dot
from numpy.linalg import norm
from tqdm import tqdm
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, AutoModel
import pickle


In [ ]:
with open('./data/ie_extracted_clinical_trials.tsv','rb') as f:
    df = pd.read_csv(f,sep='\t')
    df = df[df['eligibility_type'] == 'inclusion']

In [3]:
df['criterion'].apply(lambda x: len(x.split())).sum()

746360

In [20]:
model = AutoModel.from_pretrained("medicalai/ClinicalBERT", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("medicalai/ClinicalBERT", trust_remote_code=True, model_max_length=512, truncation=True, max_length=512)
extractor = pipeline(model=model, tokenizer=tokenizer, task="feature-extraction")

Some weights of the model checkpoint at medicalai/ClinicalBERT were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [21]:
trial2criterion = {}
trial2embd = {}
for idx, row in tqdm(df.iterrows()):
    cur_nct = row['#nct_id']
    if cur_nct not in trial2criterion:
        trial2criterion[cur_nct] = []
        trial2embd[cur_nct] = []
    trial2criterion[cur_nct].append(row['criterion'])
    
    # Extract embeddings from text row['criterion']
    cur_embd = extractor(row['criterion'], truncation=True, max_length=512)
    
    # Generate the sentence embedding
    cur_embd = np.array(cur_embd).squeeze().mean(0)
    trial2embd[cur_nct].append(cur_embd)
    

34825it [06:58, 83.26it/s] 


In [ ]:
pickle.dump(trial2criterion, open('./data/trial2criterion.pkl','wb'))
pickle.dump(trial2embd, open('./data/trial2embd.pkl','wb'))

In [25]:
trial2pool = {}
for trial in tqdm(trial2embd):
    # Get the average embedding for each trial
    trial2pool[trial] = np.array(trial2embd[trial]).mean(0)

100%|██████████| 4254/4254 [00:00<00:00, 62328.64it/s]


In [ ]:
pickle.dump(trial2pool, open('./data/trial2pool.pkl','wb'))